In [1]:
import plotly.express as px
import psycopg2
import datetime
import settings
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string

import re
nltk.download('words')
nltk.download('stopwords')
from gensim.models import KeyedVectors
from bs4 import BeautifulSoup

import logging
from numpy import random
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

[nltk_data] Downloading package words to /home/gtechzilla/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gtechzilla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#connecting to db
dbconn = psycopg2.connect("host=ec2-52-207-25-133.compute-1.amazonaws.com dbname=d8e9au4m77k9b1 user=twvlbubsgabvpj password=53cf31e1928ac9f0ec3ec5554a92bfa96ddb693b7bb3b31df2bbf3784cc66f6a")

#setting time interval from which to fetch the tweets from db
time_now = datetime.datetime.utcnow()
time_10mins_before = datetime.timedelta(hours=0,minutes=10)
print(time_10mins_before)
time_interval = time_now - time_10mins_before
print(time_interval)
#fetching tweets from db
query = "SELECT text, created_at FROM {} WHERE created_at >= '{}'".format(settings.TABLE_NAME, time_interval)
df = pd.read_sql(query,dbconn)
print(df)

0:10:00
2020-05-25 02:18:08.279263
                                                 text          created_at
0   @hopefmlive @gichuru_george All I want is you ... 2020-05-25 02:18:25
1   @C_K_Lomaria Am not around. Maliza, I will lau... 2020-05-25 02:19:01
2                     Amen and amen \n#barakazamilele 2020-05-25 02:20:31
3   Bomboclaaaaaaaaaaat! Imma slit your throat wit... 2020-05-25 02:22:57
4   Lovely and spacious 2 bedroom apartment to let... 2020-05-25 02:22:58
5   @ahmedsalims Interesting I think people don't ... 2020-05-25 02:23:20
6   Do not say, Why were the former days better th... 2020-05-25 02:24:18
7   Early to bed early to rise. Good morning World... 2020-05-25 02:25:23
8                  Good morning Liz \n#barakazamilele 2020-05-25 02:25:38
9   I say whoever cum first is loser in the game c... 2020-05-25 02:26:23
10                   Good morning the prof himuselifu 2020-05-25 02:26:28


In [3]:
words = set(nltk.corpus.words.words())

#DATA CLEANING
space_replace = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopwords = nltk.corpus.stopwords.words('english')
extensions = ['rt', 'RT']
stopwords.extend(extensions)
urls = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' 'rt')
from gensim.models.keyedvectors import KeyedVectors

#wv = api.load('word2vec-google-news-300')
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=20000)
word_vectors.init_sims(replace=True)

#function to clean the twitter text
def text_cleaning(text):
    text = BeautifulSoup(text, "lxml").text #removing any html decoding
    text = text.lower() #removing capitalization
    text = space_replace.sub(' ', text)#replacing symbols with a space
    text = bad_symbols.sub('', text) #deleting symbols from the text
    text = ' '.join(word for word in text.split() if word not in stopwords) #removing stopwords
    text = urls.sub('', text)#removing urls
    return text


#FEATURE ENGINEERING
#word averaging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [4]:
df['text'] = df['text'].astype(str)
nltk.download('punkt')
data_tokenized = df.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values
data_averaged = word_averaging_list(word_vectors,data_tokenized)

[nltk_data] Downloading package punkt to /home/gtechzilla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/gtechzilla/anaconda3/envs/nairobi_alive/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning:

Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).



In [5]:
from sklearn.externals import joblib 
  
# Load the model from the file 
depression_analyzer = joblib.load('testfile.sav')  
  
# Use the loaded model to make predictions 
df.text=df.text.astype(str)
df['status'] = depression_analyzer.predict(data_averaged) 
df.head()

/home/gtechzilla/anaconda3/envs/nairobi_alive/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

/home/gtechzilla/anaconda3/envs/nairobi_alive/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.



,text,created_at,staus
0,@hopefmlive @gichuru_george All I want is you ...,2020-05-25 02:18:25,Depressed
1,"@C_K_Lomaria Am not around. Maliza, I will lau...",2020-05-25 02:19:01,Depressed
2,Amen and amen \n#barakazamilele,2020-05-25 02:20:31,Not Depressed
3,Bomboclaaaaaaaaaaat! Imma slit your throat wit...,2020-05-25 02:22:57,Depressed
4,Lovely and spacious 2 bedroom apartment to let...,2020-05-25 02:22:58,Not Depressed


In [9]:
#plottin our pie_chart
fig = px.bar(df.status(), x="sex", y="total_bill")
fig.show()

AttributeError: 'DataFrame' object has no attribute 'status'